In [17]:
import sys
sys.path.insert(0, "../pytorchvideo")
from pytorchvideo.data import LabeledVideoDataset,Kinetics, make_clip_sampler


In [2]:
#https://www.kaggle.com/datasets/mohamedmustafa/real-life-violence-situations-dataset

In [3]:
import numpy as np
from pytorch_lightning import seed_everything, LightningModule, Trainer
import torch.nn as nn
import torch
from torch.utils.data.dataloader import DataLoader
from pytorch_lightning.callbacks import EarlyStopping,ModelCheckpoint,LearningRateMonitor
from torch.optim.lr_scheduler import CyclicLR, ReduceLROnPlateau,CosineAnnealingWarmRestarts,OneCycleLR,CosineAnnealingLR
import torchvision
import pandas as pd
import numpy as np
from glob import glob
from PIL import Image
import cv2
import os
from torch.utils.data import DataLoader, Dataset,ConcatDataset,default_collate
from sklearn.model_selection import KFold,GroupShuffleSplit,GroupKFold,LeaveOneGroupOut
from torchmetrics import MeanAbsoluteError
from sklearn.utils import shuffle
import shutil
from sklearn.model_selection import train_test_split
from torchaudio import transforms as TA
from sklearn.metrics import classification_report
import torchmetrics

In [4]:
import torch
torch.__version__

'1.11.0+cu113'

In [5]:
non=glob('NonViolence/*')
vio=glob('Violence/*')
label=[0]*len(non)+[1]*len(vio)
df=pd.DataFrame(zip(non+vio,label),columns=['file','label'])

In [6]:
train_df,val_df=train_test_split(df,test_size=0.25)

In [7]:
from pytorchvideo.data import LabeledVideoDataset,Kinetics, make_clip_sampler

from pytorchvideo.transforms import (
    ApplyTransformToKey,
    Normalize,
    RandomShortSideScale,
#     RemoveKey,
#     ShortSideScale,
    UniformTemporalSubsample,
    Permute
)

from torchvision.transforms import (
    Compose,
    Lambda,
    RandomCrop,
    RandomHorizontalFlip,
    Resize
)

from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo,
)

/home/talha/venv/lib/python3.8/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in 0.14. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/home/talha/venv/lib/python3.8/site-packages/torchvision/transforms/_transforms_video.py:25: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in 0.14. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [8]:
#tuneable params
num_video_samples=20
video_duration=2
model_name='efficient_x3d_xs'
batch_size=8
scheduler='cosine'
clipmode='random'
img_size=224

In [9]:
from pytorchvideo.data import LabeledVideoDataset, make_clip_sampler,labeled_video_dataset
from torchvision.transforms import ColorJitter,RandomAdjustSharpness,RandomAutocontrast
video_transform = Compose(
            [
            ApplyTransformToKey(
              key="video",
              transform=Compose(
                  [
                    UniformTemporalSubsample(num_video_samples),
                    Lambda(lambda x: x / 255.0),
                    Normalize((0.45, 0.45, 0.45), (0.225, 0.225, 0.225)),
                      #Determines the shorter spatial dim of the video (i.e. width or height) and scales it to the given size
                    RandomShortSideScale(min_size=img_size+16, max_size=img_size+32),
                    CenterCropVideo(img_size),
                    RandomHorizontalFlip(p=0.5),
                  ]
                ),
              ),
            ]
        )



In [10]:
train_dataset=labeled_video_dataset(val_df,
                   clip_sampler=make_clip_sampler(clipmode, video_duration),\
                    transform=video_transform, decode_audio=False
                                   )
        
train_loader=DataLoader(train_dataset,batch_size=4,
           num_workers=0,
           pin_memory=True)

In [11]:
batch=next(iter(train_loader))

In [12]:
batch['video'].shape,batch['label'].shape

(torch.Size([4, 3, 20, 224, 224]), torch.Size([4, 1]))

In [13]:
from time import sleep
import torchvision.models as models
import timm
class OurModel(LightningModule):
    def __init__(self):
        super(OurModel,self).__init__()

        self.scheduler=scheduler

        
        self.video_model = torch.hub.load('facebookresearch/pytorchvideo', model_name, pretrained=True)
        self.video_model.projection.model=nn.Linear(in_features=2048, out_features=1000, bias=True)
        
       
        self.relu=nn.ReLU()
        self.linear=nn.Linear(1000,1)
        
        self.lr=1e-3
        self.batch_size=batch_size
        self.numworker=6
        
        self.metric = torchmetrics.Accuracy()
        self.criterion=nn.BCEWithLogitsLoss()
        
    def forward(self,video):
        x=self.video_model(video)
        x=self.relu(x)
        x=self.linear(x)
        return x

    def configure_optimizers(self):
        opt=torch.optim.AdamW(params=self.parameters(),lr=self.lr )
        if self.scheduler=='cosine':
            scheduler=CosineAnnealingLR(opt,T_max=10,  eta_min=1e-6, last_epoch=-1)
            return {'optimizer': opt,'lr_scheduler':scheduler}
        elif self.scheduler=='reduce':
            scheduler=ReduceLROnPlateau(opt,mode='min', factor=0.5, patience=5)
            return {'optimizer': opt,'lr_scheduler':scheduler,'monitor':'val_loss'}
        elif self.scheduler=='warm':
            scheduler=CosineAnnealingWarmRestarts(opt,T_0=10, T_mult=1, eta_min=1e-6, last_epoch=-1)
            return {'optimizer': opt,'lr_scheduler':scheduler}
        elif self.scheduler=='cycle':
            opt=torch.optim.AdamW(params=self.parameters(),lr=1e-6 )
            scheduler=OneCycleLR(opt,max_lr=1e-2,epochs=15,steps_per_epoch=len(self.train_df)//self.batch_size//4)
            lr_scheduler = {'scheduler': scheduler, 'interval': 'step'}
            return {'optimizer': opt, 'lr_scheduler': lr_scheduler}
        elif self.scheduler=='lambda':
            lambda1 = lambda epoch: 0.9 ** epoch
            scheduler = torch.optim.lr_scheduler.LambdaLR(opt, lr_lambda=lambda1)
            return {'optimizer': opt, 'lr_scheduler': scheduler}
        elif self.scheduler=='constant':
            return opt
        
    def train_dataloader(self):
        dataset=labeled_video_dataset(train_df,
                   clip_sampler=make_clip_sampler(clipmode, video_duration),\
                    transform=video_transform, decode_audio=False)
        
        loader=DataLoader(dataset,batch_size=self.batch_size,
                   num_workers=self.numworker,
                   pin_memory=True)
        return loader

    def training_step(self,batch,batch_idx):
        video,label=batch['video'],batch['label']
#         label=label.ravel().to(torch.int64)
        out = self(video)
        loss=self.criterion(out,label)
        metric=self.metric(out,label.to(torch.int64))
        return {'loss':loss,'metric':metric.detach()}

    def training_epoch_end(self, outputs):
        loss=torch.stack([x["loss"] for x in outputs]).mean().cpu().numpy().round(2)
        metric=torch.stack([x["metric"] for x in outputs]).mean().cpu().numpy().round(2)
        self.log('train_loss', loss,batch_size=self.batch_size)
        self.log('train_metric', metric,batch_size=self.batch_size)
        print('training loss ',self.current_epoch,loss,metric)
   
    def val_dataloader(self):
        dataset=labeled_video_dataset(val_df,
                   clip_sampler=make_clip_sampler(clipmode, video_duration),\
                    transform=video_transform, decode_audio=False)
        
        loader=DataLoader(dataset,batch_size=self.batch_size,
                   num_workers=self.numworker,
                   pin_memory=True)
        return loader
    
    def validation_step(self,batch,batch_idx):
        video,label=batch['video'],batch['label']
        out = self(video)
        loss=self.criterion(out,label)
        metric=self.metric(out,label.to(torch.int64))
        return {'loss':loss,'metric':metric.detach()}

    def validation_epoch_end(self, outputs):
        loss=torch.stack([x["loss"] for x in outputs]).mean().cpu().numpy().round(2)
        metric=torch.stack([x["metric"] for x in outputs]).mean().cpu().numpy().round(2)
        print('validation loss ',self.current_epoch,loss,metric)
        self.log('val_loss', loss,batch_size=self.batch_size)
        self.log('val_metric',metric,batch_size=self.batch_size)
   
    def test_dataloader(self):
        dataset=labeled_video_dataset(val_df,
                   clip_sampler=make_clip_sampler(clipmode, video_duration),\
                    transform=video_transform, decode_audio=False)
        
        loader=DataLoader(dataset,batch_size=self.batch_size,
                   num_workers=self.numworker,
                   pin_memory=True)
        return loader

  
    def test_step(self, batch, batch_idx):
        video,label=batch['video'],batch['label']
        out = self(video)
        return { 'label': label.detach(), 'pred': out.detach()}

    def test_epoch_end(self, outputs):
        label = torch.cat([x['label'] for x in outputs]).cpu().numpy()
        pred = torch.cat([x['pred'] for x in outputs]).cpu().numpy()
        pred=np.where(pred>0.5,1,0)
        print(classification_report(label, pred))


In [14]:
checkpoint_callback = ModelCheckpoint(monitor='val_loss',dirpath='checkpoints',
                                        filename='file',save_last=True)
lr_monitor = LearningRateMonitor(logging_interval='epoch')


In [ ]:
import wandb
wandb.login()
from pytorch_lightning.loggers import WandbLogger
wandb_logger = WandbLogger(project="violence")

In [15]:
model=OurModel()
seed_everything(0)
trainer = Trainer(max_epochs=30, 
#                 deterministic=True,
                accelerator='gpu', devices=-1,
                  precision=16,
                accumulate_grad_batches=2,
                enable_progress_bar = False,
                num_sanity_val_steps=0,
                  callbacks=[lr_monitor,checkpoint_callback],
#                 limit_train_batches=5,
#                 limit_val_batches=1,
#                 logger=wandb_logger

                )

Using cache found in /home/talha/.cache/torch/hub/facebookresearch_pytorchvideo_main
Global seed set to 0
Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [16]:
trainer.fit(model,
#             ckpt_path='checkpoints/last.ckpt'
           )

Missing logger folder: /media/talha/data/image/classification/video_classification/Real Life Violence Dataset/lightning_logs
/home/talha/venv/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:611: UserWarning: Checkpoint directory /media/talha/data/image/classification/video_classification/Real Life Violence Dataset/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
Restoring states from the checkpoint path at checkpoints/last.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type              | Params
--------------------------------------------------
0 | video_model | EfficientX3d      | 5.0 M 
1 | relu        | ReLU              | 0     
2 | linear      | Linear            | 1.0 K 
3 | metric      | Accuracy          | 0     
4 | criterion   | BCEWithLogitsLoss | 0     
--------------------------------------------------
5.0 M     Trainable params
0         Non-trainable params
5.

In [ ]:
val_res=trainer.validate(model)

In [ ]:
trainer.test(model)

In [ ]:
wandb_logger.experiment.save('notebook.ipynb')